# CLI

> Command-line interface for Finnish language learning workflow

In [ ]:
#| default_exp cli

In [ ]:
#| export
import typer
from typing import Optional
from pathlib import Path
from suomi.workflow import create_cards

In [ ]:
#| export
import typer
from typing import Optional
from pathlib import Path
from suomi.workflow import create_cards

app = typer.Typer(
    help="Finnish language learning tool - create Anki flashcards with audio"
)

@app.command()
def create(
    input_file: str = typer.Argument(
        ...,
        help="Text file with Finnish words/phrases (one per line)"
    ),
    deck: str = typer.Option(
        ...,
        "--deck", "-d",
        help="Anki deck name (e.g., '06::Daily')"
    ),
    tags: str = typer.Option(
        "lang/fi",
        "--tags", "-t",
        help="Comma-separated tags (lang/fi auto-included)"
    ),
    output_dir: str = typer.Option(
        "output",
        "--output", "-o",
        help="Output directory for TSV and audio files"
    ),
):
    """
    Create Anki flashcards from Finnish texts in a file.

    The input file should contain one Finnish word or phrase per line.
    Empty lines and lines starting with # are ignored.

    This will:
    1. Read Finnish texts from input file
    2. Translate Finnish → English/Japanese (OpenAI API)
    3. Generate audio files (Piper TTS)
    4. Update TSV with media paths
    5. Upload to Anki via AnkiConnect

    Examples:
        suomi create words.txt --deck "06::Daily"
        suomi create body_parts.txt -d "05::Keho" -t "src/class,level/A1"
    """
    try:
        # Read texts from file
        input_path = Path(input_file)
        if not input_path.exists():
            typer.echo(f"❌ Error: File not found: {input_file}", err=True)
            raise typer.Exit(code=1)

        with open(input_path, encoding="utf-8") as f:
            texts = [
                line.strip()
                for line in f
                if line.strip() and not line.strip().startswith("#")
            ]

        if not texts:
            typer.echo(f"❌ Error: No texts found in {input_file}", err=True)
            raise typer.Exit(code=1)

        typer.echo(f"📖 Read {len(texts)} text(s) from {input_file}")
        typer.echo(f"🔄 Creating cards for deck: {deck}")

        create_cards(
            texts=texts,
            deck=deck,
            tags=tags,
            output_dir=output_dir
        )

        typer.echo(f"✅ Successfully created {len(texts)} card(s) in deck: {deck}")

    except Exception as e:
        typer.echo(f"❌ Error: {e}", err=True)
        raise typer.Exit(code=1)

def main():
    """Entry point for console script."""
    app()

In [ ]:
#| test
# Test: CLI help command works
from typer.testing import CliRunner
from suomi.cli import app

runner = CliRunner()
result = runner.invoke(app, ["--help"])
assert result.exit_code == 0
assert "Create Anki flashcards" in result.stdout

print("✓ CLI help test passed")

✓ CLI help test passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()